# Ensemple CB recommender based on Sparse Autoencoder

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import torch
import numpy as np
import pandas as pd

import pytorch_common.util as pu

import logging
import data as dt

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
torch.cuda.is_available()

True

In [7]:
torch.__version__

'1.11.0'

## Evaluación

In [8]:
def load(name): return pd.read_json(f'../datasets/movie_{name}_embedding.json')
def emb_rec(emb, name): return rc.DistanceMatrixRecommender(emb, f'{name}_embedding')

In [9]:
embeddings_by_name   = {name: load(name) for name in ['genres', 'overview', 'tags']}
recommenders_by_name = {name: emb_rec(emb, name) for name, emb in embeddings_by_name.items()}
emb_recommenders     = list(recommenders_by_name.values())

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

In [10]:
item_index = 10

In [11]:
recommender_group = rc.RecommenderGroup(emb_recommenders)

In [ ]:
# recommender_group.recommend(item_index).show()

In [13]:
recommender = rc.EnsembleWeightedDistanceMatrixRecommender(
    emb_recommenders,
    weights = { 'tags': 0.25, 'overview': 0.25, 'genres': 0.4}
)

In [20]:
recommender.recommend(item_index, k=10).show()


Recommender: ['genres', 'overview', 'tags']
Item


,id,title,imdb_id,image
10,8360,Shrek 2,298148,


Recommendations


,distance,title,imdb_id,image
0,0.000000,Shrek 2,298148,
1,0.006285,"Boxtrolls, The",787474,
2,0.006815,Cars,317219,
3,0.006923,Shrek,126029,
4,0.007014,Wreck-It Ralph,1772341,
5,0.007449,Snow White and the Seven Dwarfs,29583,
6,0.008184,Enchanted,461770,
7,0.008993,Sleeping Beauty,53285,
8,0.010753,Anastasia,118617,
9,0.010964,"NeverEnding Story, The",88323,Not Found Image
